In [9]:
import pandas as pd
import numpy as np
from entsoe import EntsoePandasClient

# ==========================================
# 1. CONFIGURATION
# ==========================================
API_KEY = '1efc1e14-c733-4915-bd4f-cf2e11e6750f'  # <--- Colle ta clé ici
client = EntsoePandasClient(api_key=API_KEY)

country_code = 'FR'
# On prend une période large pour avoir de l'historique
start_date = pd.Timestamp('2020-01-01', tz='Europe/Paris')
end_date = pd.Timestamp('2026-01-01', tz='Europe/Paris')

print(f"Connexion à ENTSO-E pour la zone {country_code}...")

# ==========================================
# 2. RÉCUPÉRATION DES DONNÉES BRUTES
# ==========================================

# A. PRIX DAY-AHEAD (La cible)
# Ce sont les prix fixés la veille pour le lendemain
print("   -> Téléchargement des prix...")
prices = client.query_day_ahead_prices(country_code, start=start_date, end=end_date)
df_prices = prices.to_frame(name='price_da')
# B. PRÉVISIONS DE CONSOMMATION (Load Forecast)
print("   -> Téléchargement Load Forecast...")
load = client.query_load_forecast(country_code, start=start_date, end=end_date)

# CORRECTION : Vérification du type
if isinstance(load, pd.Series):
    # Si c'est une série (liste simple), on la convertit
    df_load = load.to_frame(name='load_forecast')
else:
    # Si c'est déjà un DataFrame, on le garde tel quel et on renomme la colonne
    df_load = load.copy()
    # On force le nom de la colonne pour être sûr
    df_load.columns = ['load_forecast']

# C. PRÉVISIONS ÉOLIEN ET SOLAIRE
print("   -> Téléchargement Wind/Solar Forecast...")
generation = client.query_wind_and_solar_forecast(country_code, start=start_date, end=end_date)

# 1. On regarde ce qu'il y a dedans pour le debug (optionnel mais utile)
print(f"   Colonnes reçues : {generation.columns.tolist()}")

# 2. On renomme proprement avec un dictionnaire
# Cela évite le crash si l'ordre change
generation = generation.rename(columns={
    'Solar': 'solar_forecast',
    'Wind Onshore': 'wind_onshore',
    'Wind Offshore': 'wind_offshore'
})

# 3. On agrège tout le vent (Onshore + Offshore s'il existe)
# Si 'wind_offshore' n'existe pas, on prend juste l'onshore.
cols_wind = [c for c in generation.columns if 'wind' in c]
generation['wind_forecast'] = generation[cols_wind].sum(axis=1)

# 4. On garde uniquement les colonnes finales qu'on veut
generation = generation[['solar_forecast', 'wind_forecast']]

# ==========================================
# 3. FUSION ET NETTOYAGE (L'étape critique)
# ==========================================
print("Fusion et nettoyage...")

# Fusion sur l'index (l'heure)
df = pd.concat([df_prices, df_load, generation], axis=1)

# GESTION DES MANQUANTS
# Il arrive qu'il manque quelques heures. On interpole linéairement pour boucher les trous.
df = df.interpolate(method='linear')

# GESTION DU TIMEZONE (Vital !)
# ENTSO-E renvoie souvent de l'UTC. On s'assure d'être en heure de Paris
# pour que 12h00 corresponde bien au pic solaire.
df = df.tz_convert('Europe/Paris')

# ==========================================
# 4. FEATURE ENGINEERING (Création des variables)
# ==========================================
print("Création des features...")

# A. LA RESIDUAL LOAD (Ta variable reine)
df['res_load'] = df['load_forecast'] - (df['solar_forecast'] + df['wind_forecast'])

# B. FEATURES TEMPORELLES CYCLIQUES
# Problème : Pour un ordi, 23h et 00h sont loin (23 vs 0). En réalité, c'est collé.
# Solution : On transforme l'heure en coordonnées sinus/cosinus sur un cercle.
df['hour'] = df.index.hour
df['month'] = df.index.month

# Cycle journalier (24h)
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# Cycle annuel (12 mois)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

# C. LA CIBLE (TARGET)
# On crée la variable binaire : 1 si Prix <= 0, sinon 0
df['target_negative'] = (df['price_da'] <= 0).astype(int)

# ==========================================
# 5. VISUALISATION RAPIDE
# ==========================================
print(df[['price_da', 'res_load', 'target_negative']].head())

print(f"\nNombre d'heures à prix négatifs : {df['target_negative'].sum()}")
print(f"Pourcentage : {100 * df['target_negative'].mean():.2f}%")

# Sauvegarde pour plus tard
# df.to_csv('data_electricity_france.csv')

Connexion à ENTSO-E pour la zone FR...
   -> Téléchargement des prix...
   -> Téléchargement Load Forecast...
   -> Téléchargement Wind/Solar Forecast...
   Colonnes reçues : ['Solar', 'Wind Offshore', 'Wind Onshore']
Fusion et nettoyage...
Création des features...
                           price_da  res_load  target_negative
2020-01-01 00:00:00+01:00     41.88  64333.87                0
2020-01-01 01:00:00+01:00     38.60  62207.21                0
2020-01-01 02:00:00+01:00     36.55  61842.81                0
2020-01-01 03:00:00+01:00     32.32  58529.12                0
2020-01-01 04:00:00+01:00     30.85  56065.61                0

Nombre d'heures à prix négatifs : 3493
Pourcentage : 4.42%


In [10]:
df

AttributeError: 'Index' object has no attribute '_format_flat'

                           price_da  load_forecast  solar_forecast  \
2020-01-01 00:00:00+01:00     41.88        67300.0             0.0   
2020-01-01 01:00:00+01:00     38.60        65200.0             0.0   
2020-01-01 02:00:00+01:00     36.55        64700.0             0.0   
2020-01-01 03:00:00+01:00     32.32        61250.0             0.0   
2020-01-01 04:00:00+01:00     30.85        58650.0             0.0   
...                             ...            ...             ...   
2025-12-31 23:00:00+01:00     88.86        68500.0             0.0   
2025-12-31 23:15:00+01:00     80.19        68700.0             0.0   
2025-12-31 23:30:00+01:00     80.51        68900.0             0.0   
2025-12-31 23:45:00+01:00     71.82        68450.0             0.0   
2026-01-01 00:00:00+01:00     95.95        68450.0             0.0   

                           wind_forecast     res_load  hour  month  hour_sin  \
2020-01-01 00:00:00+01:00     2966.13000  64333.87000     0      1  0.000000   